In [1]:
import cv2
import os
import pydicom
import numpy
import SimpleITK
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
# 路径和列表声明
PathDicom = "DicomResource"  # 与python文件同一个目录下的文件夹,存储dicom文件
SaveRawDicom = "SaveRaw"     # 与python文件同一个目录下的文件夹,用来存储mhd文件和raw文件
lstFilesDCM = []
''' 
# 将PathDicom文件夹下的dicom文件地址读取到lstFilesDCM中
for dirName, subdirList, fileList in os.walk(PathDicom):
	for filename in fileList:
		if ".dcm" in filename.lower():  # 判断文件是否为dicom文件
			print(filename)
			lstFilesDCM.append(os.path.join(dirName, filename))  # 加入到列表中
'''
currentDir= os.getcwd()
print(currentDir)

# fileList = os.listdir(PathDicom)
# fileList.sort(key=lambda x:x.split('_')[1])
# lstFilesDCM = fileList
# print(fileList)
list = []
path = os.path.join(currentDir,PathDicom)
fileList = os.listdir(path)
for i in range(len(fileList)):
	list = glob(os.path.join(path,r'CT3276508-Lin ZeShan_'+'{}.DCM'.format(i+1)))
	#print(list[0])
	lstFilesDCM.append(list[0])

print(lstFilesDCM)
# for filename in fileList:
#     str = filename.split('_')[1]
#     glob(os.path.join(PathDicom,str))

In [3]:


# 第一步：将第一张图片作为参考图片，并认为所有图片具有相同维度
RefDs = pydicom.read_file(lstFilesDCM[0])  # 读取第一张dicom图片
 
# 第二步：得到dicom图片所组成3D图片的维度
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM)) # ConstPixelDims是一个元组
 
# 第三步：得到x方向和y方向的Spacing并得到z方向的层厚
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))
 
# 第四步：得到图像的原点
Origin = RefDs.ImagePositionPatient
 
# 根据维度创建一个numpy的三维数组(x,y,z)，并将元素类型设为：pixel_array.dtype
ArrayDicom = numpy.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)  # array is a numpy array

In [4]:
# 第五步:遍历所有的dicom文件，读取图像数据，存放在numpy数组中
i = 0
for filenameDCM in lstFilesDCM:
	ds = pydicom.read_file(filenameDCM)
	ArrayDicom[:, :, lstFilesDCM.index(filenameDCM)] = ds.pixel_array
	cv2.imwrite(currentDir+r'\\png\\'+"out_" + str(i) + ".png", ArrayDicom[:, :, lstFilesDCM.index(filenameDCM)])
	i += 1
 
# 第六步：对numpy数组进行转置，即把坐标轴（x,y,z）变换为（z,y,x）,这样是dicom存储文件的格式，即第一个维度为z轴便于图片堆叠
ArrayDicom = numpy.transpose(ArrayDicom, (2, 0, 1))		#原来的代码是(2,0,1),(z,x,y),也是可以的，按规范来吧，便于代码维护
 
# 第七步：将现在的numpy数组通过SimpleITK转化为mhd和raw文件
sitk_img = SimpleITK.GetImageFromArray(ArrayDicom, isVector=False)
sitk_img.SetSpacing(ConstPixelSpacing)
sitk_img.SetOrigin(Origin)
SimpleITK.WriteImage(sitk_img, os.path.join(currentDir,SaveRawDicom, "sample" + ".mhd"))

In [5]:
# # 显示原始dicom文件的代码
# image_dicom =SimpleITK.ReadImage(lstFilesDCM)
# image_dicom = SimpleITK.GetArrayFromImage(image_dicom)
# #image_mhd = np.squeeze(image_mhd[slice, ...])  # if the image_mhd is 3d, the slice is integer	
# for i in range(len(image_dicom)):
#     plt.imshow(image_dicom[i,:,:],cmap='gray')
#     plt.axis('off')
#     plt.show()

In [ ]:
# 测试生成的mhd图像文件,mhd是(z,x,y)
image_mhd =SimpleITK.ReadImage('./SaveRaw/sample.mhd')
image_mhd = SimpleITK.GetArrayFromImage(image_mhd)
#image_mhd = np.squeeze(image_mhd[slice, ...])  # if the image_mhd is 3d, the slice is integer
#image_mhd = numpy.transpose(ArrayDicom, (0, 2, 1))	
for i in range(len(image_mhd)):
    plt.imshow(image_mhd[i,:,:],cmap='gray')
    plt.axis('off')
    plt.show()
#cv2.imwrite('1.png',image_mhd)